# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [ ]:
DROP TABLE IF EXISTS rentals_may;

CREATE TABLE rentals_may LIKE rental;

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [ ]:
INSERT INTO rentals_may
SELECT * FROM rental
WHERE MONTH(rental_date) = 05;

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [ ]:
DROP TABLE IF EXISTS rentals_june;

CREATE TABLE rentals_june LIKE rental;

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [ ]:
INSERT INTO rentals_june
SELECT * FROM rental
WHERE MONTH(rental_date) = 06;

## Check the number of rentals for each customer for May.


In [ ]:
SELECT customer_id, COUNT(rental_id) AS may_count
FROM rentals_may
GROUP BY customer_id;

## Check the number of rentals for each customer for June.
 

In [ ]:
SELECT customer_id, COUNT(rental_id) AS june_count
FROM rentals_june
GROUP BY customer_id;

## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

In [ ]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [ ]:
rentalcount_may = pd.read_sql_query(
    '''
    SELECT customer_id, COUNT(rental_id) AS may_count
    FROM rentals_may
    GROUP BY customer_id;
    ''', engine)
    
rentalcount_june = pd.read_sql_query(
    '''
    SELECT customer_id, COUNT(rental_id) AS june_count
    FROM rentals_june
    GROUP BY customer_id;
    ''', engine)

## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [ ]:
rentalcount_mayjune = pd.merge(rentalcount_may, rentalcount_june, how='outer')
rentalcount_mayjune.head()

In [ ]:
def may_vs_june(id): 
    if (rentalcount_mayjune['customer_id'] == id).any() == False:
        print('There is no customer with this id.')
    elif (rentalcount_mayjune['customer_id'] == id).any() == True:
        for row in range(len(rentalcount_mayjune)):
            if rentalcount_mayjune['customer_id'][row] != id:
                pass
            elif rentalcount_mayjune['customer_id'][row] == id:
                print('Customer', id, 'rented', round(rentalcount_mayjune['may_count'][row]), 'films in May.')
                print('Customer', id, 'rented', round(rentalcount_mayjune['june_count'][row]), 'films in June.')
                if rentalcount_mayjune['may_count'][row] > rentalcount_mayjune['june_count'][row]:
                    print('Customer', id, 'rented more films in May than in June.')
                elif rentalcount_mayjune['may_count'][row] < rentalcount_mayjune['june_count'][row]:
                    print('Customer', id, 'rented more films in June than in May.')
                else:
                    'Customer', id, 'rented the same amount of films in May and in June.'
    return id 

In [ ]:
may_vs_june(67)